In [4]:
import requests
from datetime import datetime
import time

# 最接近的只有一小时数据

API_KEY = "CG-men8aqqgPEPpKAfCmtNvmGrN"
coins = ["bitcoin", "ethereum", "litecoin"]  # List of coin IDs (as used by CoinGecko)

def get_historical_prices(coin_id, target_time):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"

    # Define a wider time range: 5 seconds before and after
    from_timestamp = target_time - (3*3600)
    to_timestamp = target_time + (3*3600)

    # Parameters for the API request
    params = {
        "vs_currency": "usd",
        "from": from_timestamp,
        "to": to_timestamp,
    }

    # Headers with API key
    headers = {
        "accept": "application/json",
        "x-cg-demo-api-key": API_KEY
    }

    try:
        # Make the API request
        response = requests.get(url, params=params, headers=headers)

        # Check if the request was successful
        response.raise_for_status()

        # Parse the JSON response
        data = response.json()

        # Check if price data is available
        if "prices" in data and data["prices"]:
            # Calculate time differences and store price data
            target_time_ms = target_time * 1000  # Convert to milliseconds
            price_data = [
                {
                    "timestamp_ms": timestamp_ms,
                    "price": price,
                    "time_diff": abs(timestamp_ms - target_time_ms)
                }
                for timestamp_ms, price in data["prices"]
            ]

            # Sort by time difference and get the 4 closest
            price_data_sorted = sorted(price_data, key=lambda x: x["time_diff"])[:4]

            return price_data_sorted
        else:
            print(f"Error: No price data available for {coin_id} in the specified time range.")
            return None

    except requests.exceptions.HTTPError as http_err:
        if response.status_code == 429:
            print("Error: Rate limit exceeded. Please wait and try again later.")
        else:
            print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError:
        print("Error: Failed to connect to the CoinGecko API. Check your internet connection.")
    except requests.exceptions.Timeout:
        print("Error: The request timed out. Please try again later.")
    except requests.exceptions.RequestException as err:
        print(f"An error occurred: {err}")
    except KeyError:
        print("Error: Unexpected response format from the API.")

    return None  # Return None if any error occurred

# Main Execution
target_time = int(datetime(2025, 3, 19, 16, 35).timestamp())  # Same target time as before

for coin_id in coins:
    prices = get_historical_prices(coin_id, target_time)

    if prices:
        print(f"\nFour closest {coin_id.capitalize()} price data points to March 19, 2025, 16:35 UTC:")
        for idx, entry in enumerate(prices, 1):
            timestamp_ms = entry["timestamp_ms"]
            price = entry["price"]
            readable_time = datetime.utcfromtimestamp(timestamp_ms / 1000).strftime("%Y-%m-%d %H:%M:%S UTC")
            print(f"{idx}. Time: {readable_time}, Price (USD): ${price:,.2f}")



Four closest Bitcoin price data points to March 19, 2025, 16:35 UTC:
1. Time: 2025-03-19 16:09:55 UTC, Price (USD): $84,620.39
2. Time: 2025-03-19 17:04:50 UTC, Price (USD): $84,590.97
3. Time: 2025-03-19 18:04:48 UTC, Price (USD): $84,258.56
4. Time: 2025-03-19 15:04:47 UTC, Price (USD): $84,218.78

Four closest Ethereum price data points to March 19, 2025, 16:35 UTC:
1. Time: 2025-03-19 16:09:44 UTC, Price (USD): $2,045.45
2. Time: 2025-03-19 17:04:50 UTC, Price (USD): $2,047.97
3. Time: 2025-03-19 18:04:40 UTC, Price (USD): $2,027.44
4. Time: 2025-03-19 15:04:23 UTC, Price (USD): $2,030.58

Four closest Litecoin price data points to March 19, 2025, 16:35 UTC:
1. Time: 2025-03-19 16:09:41 UTC, Price (USD): $91.95
2. Time: 2025-03-19 17:05:23 UTC, Price (USD): $91.58
3. Time: 2025-03-19 18:04:58 UTC, Price (USD): $91.14
4. Time: 2025-03-19 15:03:34 UTC, Price (USD): $91.93
